In [2]:
import os
import shutil 
import numpy as np
from PIL import Image

Create subdirectories in VAL folder


In [6]:
trainPath = r'C:\Users\rrayn\OneDrive\Desktop\Personal-Project\Brain MRI Analysis\Brain-MRI-Analysis\Br35H-Mask-RCNN\TRAIN'
valPath = r'C:\Users\rrayn\OneDrive\Desktop\Personal-Project\Brain MRI Analysis\Brain-MRI-Analysis\Br35H-Mask-RCNN\VAL'
classes = ['yes', 'no']

for className in classes:
    os.makedirs(os.path.join(valPath, className), exist_ok=True)

Move some data from train to val

In [7]:
valPercentage = 0.2

for className in classes:
    trainClassPath = os.path.join(trainPath , className)
    valClassPath = os.path.join(valPath, className)
    allFiles = os.listdir(trainClassPath)
    np.random.shuffle(allFiles)
    splitIndex = int(valPercentage * len(allFiles))
    valFiles = allFiles[:splitIndex]
    for fileName in valFiles:
        src = os.path.join(trainClassPath, fileName)
        dst = os.path.join(valClassPath, fileName)
        shutil.move(src,dst)

Change image color channels to RGB 


In [4]:
trainPath = r'C:\Users\rrayn\OneDrive\Desktop\Personal-Project\Brain MRI Analysis\Brain-MRI-Analysis\Br35H-Mask-RCNN\TRAIN'
valPath = r'C:\Users\rrayn\OneDrive\Desktop\Personal-Project\Brain MRI Analysis\Brain-MRI-Analysis\Br35H-Mask-RCNN\VAL'
for root, dirs, files in os.walk(trainPath):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                img_path = os.path.join(root, file)
                img = Image.open(img_path)
                img_rgb = img.convert('RGB')
                img_rgb.save(img_path)  # Overwrite the original image

for root, dirs, files in os.walk(valPath):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                img_path = os.path.join(root, file)
                img = Image.open(img_path)
                img_rgb = img.convert('RGB')
                img_rgb.save(img_path)  # Overwrite the original image